# Training a classifier

## Imports

In [1]:
import torch
import torchvision
import torchvision.transforms as transforms

## 1. Load and Normalizing the data

In [2]:
transform = transforms.Compose([
    transforms.ToTensor()
    ,transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])

trainset = torchvision.datasets.CIFAR10(
    root = './data'
    , train = True
    , download = True
    , transform = transform
)

trainloader = torch.utils.data.DataLoader(
    dataset = trainset
    , batch_size = 4
    , shuffle = True
    , num_workers = 2
)

testset = torchvision.datasets.CIFAR10(
    root = './data'
    , train = False
    , download = True
    , transform = transform
)

testloader = torch.utils.data.DataLoader(
    dataset = testset
    , batch_size = 4
    , shuffle = False
    , num_workers = 2
)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


## 2. Data exploration

In [3]:
import matplotlib.pyplot as plt
import numpy as np

# function to show an image

def imshow(img):
    img = img / 2 + 0.5 # unnormalize
    npimg = img.numpy() 
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    
# get some random training images
dataiter = iter(trainloader)
images, labels = dataiter.next()

# show images
imshow(torchvision.utils.make_grid(images))
# print labels
print(' '.join('%5s' % classes[labels[j]] for j in range(4)))

plane   dog  ship  bird


## 3. Bulding a convolutional  NN

In [4]:
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
net = Net()

## 4.Define loss function

In [5]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr = 0.001, momentum = 0.9)

## 5.Train the NN

In [6]:
from datetime import datetime
start = datetime.now()

for epoch in range(2):
    running_loss = 0.0
    for i, data in  enumerate(trainloader, 0):
        # get the input
        inputs, labels = data
        
        # wrap them in variable
        inputs, labels = Variable(inputs), Variable(labels)
        
        # zero the parameter gradients
        optimizer.zero_grad()
        
        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        # print statistics
        running_loss += loss.data[0]
        if i % 2000 == 1999:
            print('[%d, %5d] loss: %.3f' %
                 (epoch + 1, i + 1, running_loss / 2000)
                 )
            running_loss = 0

            
finish = datetime.now()
elapsed = finish - start
print('Finished Training in: ' + str(elapsed))
        
    

[1,  2000] loss: 2.165
[1,  4000] loss: 1.863
[1,  6000] loss: 1.711
[1,  8000] loss: 1.604
[1, 10000] loss: 1.541
[1, 12000] loss: 1.468
[2,  2000] loss: 1.430
[2,  4000] loss: 1.374
[2,  6000] loss: 1.346
[2,  8000] loss: 1.329
[2, 10000] loss: 1.303
[2, 12000] loss: 1.255
Finished Training in: 0:01:11.412798
